<a href="https://www.kaggle.com/code/ayushs9020/utc-training-pytorch-dataloader-cafa?scriptVersionId=135578264" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# CAFA 5 Protein Prediction

<img src = "https://i.pinimg.com/originals/8e/de/73/8ede737acdcf671e000ae0f87a742e40.png" width = 400>

The `goal` of this competition is to `predict the function of a set of proteins`. We will `develop a model trained` on the `amino-acid sequences` of the `proteins and on other data`. Our work `will help researchers` better `understand the function of proteins`, which is `important for discovering` `how cells, tissues, and organs work`.

# 1 | Basic Terminologies 💻

* $Structure$ $of$ $a$ $Protein$
* $Gene$ $Ontology$ $(GO)$

## 1.1 | Structure of a Protein

So what is a actually a **Protein...?**

First of all lets understand the structure of an **Atom**

<img src = "https://www.sciencefacts.net/wp-content/uploads/2020/11/Parts-of-an-Atom-Diagram.jpg"  width = 300>

There is a really good image I found of the `structure of atom`. Though there are many debates on the structure like this, but this `model is accepted universaly at this moment`.

In the centre we have the `Neucleus`. The `Neucleus` is made up of $2$ more structures named as `Neutron` and `Proton`. A `Proton` is `positively charged element` and a `Neutron` is a `neutral charged element`. A `Electron`, `negatively charged element`, `orbits` this `Neucleus` at some `distance apart`.

The `more we increase the number` of `Electrons` and `Protons`. The `bigger the atoms becomes`.

There are `different shells` where the `Electrons reside`. The `more closer the shell` is, the `less Electrons` it contrains. There are mainly $4$ shells. 

|||
|---|---
|$K$|$2$
|$L$|$8$
|$M$|$18$
|$N$|$32$

Once an atom `fills its outer most shell` with `Electrons`. It becomes `stable atom` and try to `refuse any donation` or `recieve of extra atom`.

<img src = "https://cdn1.byjus.com/wp-content/uploads/2022/01/word-image128.png" width = 400>

`Different atoms combine` to `share Electrons` and become `stable Molecules` 

<img src = "https://www.astrochem.org/sci_img/Amino_Acid_Structure.jpg" width = 300>

A `Amino Acid` is made up of mainly $4$ different atoms
`[H , C , O , N]`. 

<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/5/51/L-amino_acid_structure.svg/1200px-L-amino_acid_structure.svg.png" alt = "Bro use Light Theme" width = 300 >

We also have a free `Electron Pair` of `Carbon` in this molecule, we call this as a `Side Chain` which can be of different types. Basically this `Side Chain` provide the flexibility to make `different types` of `Amino Acids`. This flexibilty allows for $20$ `different` `Amino Acids` 

When we join `Amino Acids` with `peptide bonds`, we get `Proteins`. Conncecting different types of `Amino Acids` ends up in different types of `Proteins`.

## 1.2 | Gene Ontology (GO)

$Gene$ $Ontology$ $(GO)$ is a `controlled vocabulary` that `describes the functions of genes` and gene products. It is constantly being updated as new information becomes available.

There are mainly $3$ `Ontologies`

||||
|---|---|---
|$Biological$ $Process$|Describes the `biological processes`|A gene product might be involved in the process of `cell cycle`/`signal transduction.`
|$Cellular$ $Component$|Describes the `cellular components`|A gene product might be located in the `nucleus`/`cytoplasm`.
|$Molecular$ $Function$|Describes the `molecular functions`|A gene product might be involved in the `catalysis of a reaction`/`binding of a molecule`.

**[Gene Ontology Documentation](http://geneontology.org/docs/ontology-documentation/)**

# 2 | Data 📊

In [1]:
import pandas as pd 
import re
from Bio.Seq import Seq

The $Training$ $Set$ contains all `proteins with annotated terms` that have been validated by 
* $Experimental$
* $High-Throughput$ $Evidence$
* [$Traceable$ $Author$ $Statement$](https://wiki.geneontology.org/index.php/Traceable_Author_Statement_(TAS)#:~:text=The%20TAS%20evidence%20code%20covers,annotations%20come%20from%20review%20articles.)
* [$Inferred$ $by$ $Curator$ $(IC)$](https://wiki.geneontology.org/Inferred_by_Curator_(IC)) 

**Any other sources of Data are allowed**

### 2.1.1.1 | Go-Basic.obo

The $Ontology$ data is in the `file go-basic.obo`. This file is in $OBO$ `Biology-Oriented Language`. The nodes in `the graph are indexed` by the `term name`
```
subontology_roots = {'BPO':'GO:0008150',
                     'CCO':'GO:0005575',
                     'MFO':'GO:0003674'}
```

In [2]:
with open('/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo') as file :
    
    content = file.read()
    stanzas =  re.findall(r'\[Term\][\s\S]*?(?=\n\[|$)' , content)
    
print(stanzas[0])

[Term]
id: GO:0000001
name: mitochondrion inheritance
namespace: biological_process
def: "The distribution of mitochondria, including the mitochondrial genome, into daughter cells after mitosis or meiosis, mediated by interactions between mitochondria and the cytoskeleton." [GOC:mcc, PMID:10873824, PMID:11389764]
synonym: "mitochondrial inheritance" EXACT []
is_a: GO:0048308 ! organelle inheritance
is_a: GO:0048311 ! mitochondrion distribution



### 2.1.1.2 | Training Sequences.fasta

This file contains only `sequences` for `proteins` with `annotations` in the dataset `labeled proteins`.

This files are in `FASTA` format. 

This file contains . To obtain the full set of protein sequences for unlabeled proteins, the Swiss-Prot and TrEMBL databases can be found here.

In [3]:
with open("/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta" , "r") as file:
    x = file.read()
    
x[:200]

'>P20536 sp|P20536|UNG_VACCC Uracil-DNA glycosylase OS=Vaccinia virus (strain Copenhagen) OX=10249 GN=UNG PE=1 SV=1\nMNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIPDKFFIQLKQPLRNK\nRVCVCGIDPYPKDGTGVPFESPNF'

### 2.1.1.3 | Train Terms.tsv

This file contains the list of annotated terms `ground truth` for the proteins in `train_sequences.fasta`. 

In [4]:
pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv" , sep = "\t")

,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO
...,...,...,...
5363858,X5L565,GO:0050649,MFO
5363859,X5L565,GO:0016491,MFO
5363860,X5M5N0,GO:0005515,MFO
5363861,X5M5N0,GO:0005488,MFO


The first column indicates the `protein's UniProt accession ID`, the second is the `GO term ID`, and the third indicates in which `ontology the term appears`.

### 2.1.1.4 | Train Taxonomy.tsv

This file contains the list of `proteins and the species to which they belong`, represented by a `taxonomic identifier` `taxon ID` number.

In [5]:
pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv" , sep = "\t")

,EntryID,taxonomyID
0,Q8IXT2,9606
1,Q04418,559292
2,A8DYA3,7227
3,Q9UUI3,284812
4,Q57ZS4,185431
...,...,...
142241,Q5TD07,9606
142242,A8BB17,7955
142243,A0A2R8QBB1,7955
142244,P0CT72,284812


### 2.1.1.5 | IA.txt

IA.txt contains the information accretion (weights) for each GO term. These weights are used to compute weighted precision and recall, as described in the Evaluation section. 

## 2.1.2 | Test Set

The $Test$ $Set$ is `unknown at the beginning` of the competition. It will contain `protein sequences` `their functions` from the `test superset` that `gained experimental annotations` between the `submission-deadline` and the `time of evaluation`.

# 3 | PyTorch DataLoader ⚙️

In [6]:
import numpy as np

import torch
from torch.utils.data import Dataset

The dataloader is highly inspired by **[Henri Upton](https://www.kaggle.com/henriupton)=>[ProteiNet 🧬 PyTorch+EMS2/T5/ProtBERT Embeddings](https://www.kaggle.com/code/henriupton/proteinet-pytorch-ems2-t5-protbert-embeddings)**

First we will make a simple class...

In [7]:
class P_Dataset(Dataset):
    pass

Now we will load the `embeds` from the `T5 Embeds`

Embeds are like representation of a non-numercial elemenet to a list of numerical elemenet. 

In [8]:
class P_Dataset(Dataset):
    
    def __init__(self):
        
        super(P_Dataset).__init__()
        
        self.embeds = np.load("/kaggle/input/t5embeds/test_embeds.npy")

In case you wanna see, what these `embeds`looks like, open the below hidden cells

In [9]:
embeds = np.load("/kaggle/input/t5embeds/test_embeds.npy")

embeds , embeds.shape

(array([[ 0.05470492,  0.06342026, -0.01531996, ..., -0.04331931,
          0.03600927,  0.06309301],
        [ 0.09037268,  0.08984205, -0.02388695, ..., -0.05335043,
          0.01964429,  0.07962959],
        [ 0.04358805,  0.03957234, -0.01433173, ..., -0.04446448,
          0.03097377,  0.04032155],
        ...,
        [ 0.03274843,  0.14186755,  0.03414193, ...,  0.0206462 ,
          0.05467706, -0.01504807],
        [ 0.05271317,  0.15701264,  0.04327936, ...,  0.02159062,
          0.0625835 , -0.01490347],
        [ 0.04187706,  0.13404095,  0.0790938 , ...,  0.0541121 ,
          0.01576663, -0.02881737]]),
 (141865, 1024))

For the `training` we need the `targets`, which we will take from the `train_targets_top500.npy`.

In [10]:
class P_Dataset(Dataset):
    
    def __init__(self , datatype):
        
        super(P_Dataset).__init__()
        
        self.datatype  = datatype
        
        self.embeds = np.load("/kaggle/input/t5embeds/test_embeds.npy")
        
        if self.datatype == "train" : 
            
            self.targets = np.load("/kaggle/input/train-targets-top500/train_targets_top500.npy")[:self.embeds.shape[0]]

These are our targets

In [11]:
targets = np.load("/kaggle/input/train-targets-top500/train_targets_top500.npy")[:embeds.shape[0]]

targets

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

Now we will just apply some `Getters` in the class 

In [12]:
class P_Dataset(Dataset):

    def __init__(self , datatype ):
        super(P_Dataset).__init__()

        self.datatype = datatype
        self.embeds = np.load("/kaggle/input/t5embeds/test_embeds.npy")
        
        if datatype == "train":

            self.targets = np.load("/kaggle/input/train-targets-top500/train_targets_top500.npy")[:self.embeds.shape[0]]
            
    def __len__(self):return self.targets.shape[0]

    def __getitem__(self , index):

        r_embed = torch.tensor(self.embeds[index] , dtype = torch.float32)

        if self.datatype == "train":

            r_targets = torch.tensor(self.targets[index], dtype = torch.float32)

            return r_embed, r_targets

        return r_embed

In [13]:
train_data = P_Dataset(datatype = "train")

In [14]:
train_dataloader = torch.utils.data.DataLoader(train_data , batch_size = 128 , shuffle = True)

# 4 | Model Setup 

Now we will train Simple models, and see the results

# 4.1 | MultiLayerPerceptron 

A `MultiLayerPerceptron` is a simple model, which consists of just few perceptrons/nodes/neurons connected with each other. 

As we have choosen for the `top-500` targets, our last layer of the network will contain 500 perceptrons, 

In [15]:
class MultiLayerPerceptron(torch.nn.Module):

    def __init__(self):
        super(MultiLayerPerceptron, self).__init__()

        self.linear1 = torch.nn.Linear(1024 , 1012)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(1012 , 712)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(712 , 500)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        return x

I cannot use `Kaggle GPUs` for some reason (I dont know why), so I will be commenting the `cuda` parts 

In [16]:
MLP = MultiLayerPerceptron()

# MLP = MultiLayerPerceptron().to("cuda")

# 5 | Loss Function 

$Loss$ is a `measure` of `how well a model is performing` on a given task. It is `calculated` by `comparing` the `model's predictions` to the `ground truth labels`. The `lower the loss`, the `better the model` is performing.

Here we will be using `Cross Entropy Loss` , a loss function that `measures` the `difference` between the `model's predicted probability distribution` and the `ground truth distribution`.

In [17]:
CrossEntropy = torch.nn.CrossEntropyLoss()

# 6 | Optimizer 

An $Optimizer$ is an `algorithm` or function that `updates the weights and biases` of a neural network in order to `minimize a loss function`. 

Here we will be using the `Adam Optimizer`

$Adam$ is an $Adaptive$ $Learning$ $Rate$ method, which works by `maintaining two moving averages of the gradients`
* $Mean$ - Calculate the `momentum term`, which helps to `prevent` the `optimizer` from `getting stuck in local minima`
* $Variance$ - Calculate the `learning rate`, which is `adjusted based on the magnitude of the gradients`.

$$m_t = \beta_1 * m_{t - 1} + (1 - \beta_1) * w_t$$

$$v_t = \beta_2 * m_{t - 1} + (1 - \beta_2) * w_t$$

$$m_t = \frac{m_t}{1 - \beta_1^t}$$

$$v_t = \frac{v_t}{1 - \beta_2^t}$$

$$w_{t+1} = w_t - \frac{n}{\sqrt{v_t + e}} * m_t$$

In [18]:
optimizer = torch.optim.Adam(MLP.parameters(), lr = 0.0005)

# 7 | Training Loop  

In [19]:
from IPython.display import IFrame

Now we will start the training loop 

I dont the exact reason, but everytime I try to access `GPU` for some training in `Kaggle`. `CUDA goes out of memory`. Thus I have trained the model on `Colab` and will imported the results to `Wandb`.

```
los = []

for epochs in (range(5)):

    losses = []

    for x , y in tqdm.tqdm(train_dataloader):
        
        x = torch.tensor(x , dtype = torch.float32)
        y = torch.tensor(x , dtype = troch.float32)
        
        
#         x = torch.tensor(x , dtype = torch.float32).to("cuda")
#         y = torch.tensor(y , dtype = torch.float32).to("cuda")

        optimizer.zero_grad()
        preds = model(x)

        loss = CrossEntropy(preds, y)
        losses.append(loss)

    los.append(losses)
```

```
---------------------------------------------------------------------------
OutOfMemoryError                          Traceback (most recent call last)
Cell In[31], line 15
     12 img = img.to("cuda")
     13 mask = mask.to("cuda")
---> 15 outputs = model(img)  
     17 loss =  loss_func(outputs , mask)
     19 loss.backward()

File /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501, in Module._call_impl(self, *args, **kwargs)
   1496 # If we don't have any hooks, we want to skip the rest of the logic in
   1497 # this function, and just call forward.
   1498 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1499         or _global_backward_pre_hooks or _global_backward_hooks
   1500         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1501     return forward_call(*args, **kwargs)
   1502 # Do not call functions when jit is used
   1503 full_backward_hooks, non_full_backward_hooks = [], []

File /opt/conda/lib/python3.10/site-packages/segmentation_models_pytorch/base/model.py:29, in SegmentationModel.forward(self, x)
     25 """Sequentially pass `x` trough model`s encoder, decoder and heads"""
     27 self.check_input_shape(x)
---> 29 features = self.encoder(x)
     30 decoder_output = self.decoder(*features)
     32 masks = self.segmentation_head(decoder_output)

File /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501, in Module._call_impl(self, *args, **kwargs)
   1496 # If we don't have any hooks, we want to skip the rest of the logic in
   1497 # this function, and just call forward.
   1498 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1499         or _global_backward_pre_hooks or _global_backward_hooks
   1500         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1501     return forward_call(*args, **kwargs)
   1502 # Do not call functions when jit is used
   1503 full_backward_hooks, non_full_backward_hooks = [], []

File /opt/conda/lib/python3.10/site-packages/segmentation_models_pytorch/encoders/efficientnet.py:73, in EfficientNetEncoder.forward(self, x)
     71             drop_connect = drop_connect_rate * block_number / len(self._blocks)
     72             block_number += 1.0
---> 73             x = module(x, drop_connect)
     75     features.append(x)
     77 return features

File /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501, in Module._call_impl(self, *args, **kwargs)
   1496 # If we don't have any hooks, we want to skip the rest of the logic in
   1497 # this function, and just call forward.
   1498 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1499         or _global_backward_pre_hooks or _global_backward_hooks
   1500         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1501     return forward_call(*args, **kwargs)
   1502 # Do not call functions when jit is used
   1503 full_backward_hooks, non_full_backward_hooks = [], []

File /opt/conda/lib/python3.10/site-packages/efficientnet_pytorch/model.py:111, in MBConvBlock.forward(self, inputs, drop_connect_rate)
    109 x = self._depthwise_conv(x)
    110 x = self._bn1(x)
--> 111 x = self._swish(x)
    113 # Squeeze and Excitation
    114 if self.has_se:

File /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501, in Module._call_impl(self, *args, **kwargs)
   1496 # If we don't have any hooks, we want to skip the rest of the logic in
   1497 # this function, and just call forward.
   1498 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1499         or _global_backward_pre_hooks or _global_backward_hooks
   1500         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1501     return forward_call(*args, **kwargs)
   1502 # Do not call functions when jit is used
   1503 full_backward_hooks, non_full_backward_hooks = [], []

File /opt/conda/lib/python3.10/site-packages/efficientnet_pytorch/utils.py:80, in MemoryEfficientSwish.forward(self, x)
     79 def forward(self, x):
---> 80     return SwishImplementation.apply(x)

File /opt/conda/lib/python3.10/site-packages/torch/autograd/function.py:506, in Function.apply(cls, *args, **kwargs)
    503 if not torch._C._are_functorch_transforms_active():
    504     # See NOTE: [functorch vjp and autograd interaction]
    505     args = _functorch.utils.unwrap_dead_wrappers(args)
--> 506     return super().apply(*args, **kwargs)  # type: ignore[misc]
    508 if cls.setup_context == _SingleLevelFunction.setup_context:
    509     raise RuntimeError(
    510         'In order to use an autograd.Function with functorch transforms '
    511         '(vmap, grad, jvp, jacrev, ...), it must override the setup_context '
    512         'staticmethod. For more details, please see '
    513         'https://pytorch.org/docs/master/notes/extending.func.html style="color:rgb(175,0,0)">')

File /opt/conda/lib/python3.10/site-packages/efficientnet_pytorch/utils.py:67, in SwishImplementation.forward(ctx, i)
     65 @staticmethod
     66 def forward(ctx, i):
---> 67     result = i * torch.sigmoid(i)
     68     ctx.save_for_backward(i)
     69     return result

OutOfMemoryError: CUDA out of memory. Tried to allocate 40.00 MiB (GPU 0; 15.90 GiB total capacity; 319.37 MiB already allocated; 7.75 MiB free; 326.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
```

In [20]:
IFrame("https://wandb.ai//ayushsinghal659/CAFA/reports/CAFA-Multi-Layer-Perceptron--Vmlldzo0NzgyMTgy" , 1300 , 400)

As we can see we did not get good results, but we will try to omprove our results, by improving the model and by adding new one 

# 8 | TO DO LIST 📄

```
TO DO 1 : VISUALIZE THE DATA

TO DO 2 : TRAIN A MODEL

TO DO 3 : TRY DIFFERENT MODELS

TO DO 4 : ADD WANDB SUPPORT

TO DO 5 : ADD TENSORFLOW DATA LOADER

TO DO 6 : TRAIN A TF MODEL

TO DO 7 : IMPROVE RESULTS

TO DO 8 : DECREASE TRAINING TIME

TO DO 9 : DANCE 
```

# 9 | Ending 🏁

**THAT'S IT FOR TODAY GUYS**

**WE WILL GO DEEPER INTO THE DATA IN THE UPCOMING VERSIONS**

**PLEASE COMMENT YOUR THOUGHTS, HIHGLY APPRICIATED**

**DONT FORGET TO MAKE AN UPVOTE, IF YOU LIKED MY WORK $:)$**

<img src = "https://i.imgflip.com/19aadg.jpg">

**PEACE OUT $:)$**